## Trabajo práctico 1: Bayes Ingenuo
---
*Estudiantes:* 
- Dennis Luna
- Graciela Rivera
- Luis E. Vargas 
---

## *I Sección: Implementación de la clasificación multi-clase con Bayes ingenuo*

### Librerías

In [581]:
import torch
import torchvision
from PIL import Image
import torchvision.transforms.functional as TF
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd

### 1. Carga de las imágenes- mnist_dataset

In [582]:

def binarize_image(image_tensor):
    image_tensor[image_tensor > 0.5] = 1
    image_tensor[image_tensor <= 0.5] = 0
    return image_tensor

def imshow(image_tensor):
    im = plt.imshow(image_tensor.numpy(), cmap = 'gray')
    plt.show()
       
def load_single_image(path = "src_base/mnist_dataset/train", img_index=1, mode="bin"):
    #Open up the dataset
    dataset =  torchvision.datasets.ImageFolder(path)   
    #print(dataset)
    list_images = dataset.imgs
    pair_path_label = list_images[img_index]
    new_image = Image.open(pair_path_label[0]) 
    new_image_plain = None
    
    if(mode == "bin"):
        new_image_tf = TF.to_tensor(new_image).squeeze() #Return tensor in range [0.0 - 1.0]
        new_image_bin = binarize_image(new_image_tf)
        new_image_plain = new_image_bin.view(new_image_bin.shape[0] * new_image_bin.shape[1])
    elif(mode == "gray"):
        new_image_gray = TF.pil_to_tensor(new_image).squeeze() #Return tensor in grayscale
        new_image_plain = new_image_gray.view(new_image_gray.shape[0] * new_image_gray.shape[1])
    else:
        print("\n-E- Modo seleccionado invalido\n")
        
    return(new_image_plain)
        
def load_dataset(path = "src_base/mnist_dataset/train"):
    #Open up the dataset
    dataset =  torchvision.datasets.ImageFolder(path)   
    #print(dataset)
    list_images = dataset.imgs
    #print(list_images)
    train_data_tensor_bin  = None 
    train_data_tensor_gray = None
    labels_training = []
    first_tensor = True
    #list_images_training =  set(data_labeled.train_ds.x.items)
    #print(list_images)
    for i in range(len(list_images)):
        pair_path_label = list_images[i]
        image = Image.open(pair_path_label[0])        

        x_tensor = TF.to_tensor(image).squeeze() #Return tensor in range [0.0 - 1.0]
        x_tensor_gray = TF.pil_to_tensor(image).squeeze() #Return tensor in grayscale
        x_tensor_bin = binarize_image(x_tensor)
               
        #plt.figure()
        #plt.imshow(x_tensor_bin)
        x_tensor_bin_plain = x_tensor_bin.view(x_tensor_bin.shape[0] * x_tensor_bin.shape[1], -1)
        x_tensor_gray_plain = x_tensor_gray.view(x_tensor_gray.shape[0] * x_tensor_gray.shape[1], -1)
        #print("tensor ", x_tensor_bin_plain)
        #test dataset case        
        #if("train" in pair_path_label[0]):
        labels_training += [pair_path_label[1]]
        #print(pair_path_label)
        if(first_tensor):
            first_tensor = False
            train_data_tensor_bin = x_tensor_bin_plain
            train_data_tensor_gray = x_tensor_gray_plain
        else:
            train_data_tensor_bin = torch.cat((train_data_tensor_bin, x_tensor_bin_plain), 1)  
            train_data_tensor_gray = torch.cat((train_data_tensor_gray, x_tensor_gray_plain), 1)
    return (train_data_tensor_bin, train_data_tensor_gray, torch.tensor(labels_training))       

(train_data_tensor_bin, train_data_tensor_gray, labels_training) = load_dataset()

print("train gray dimensions", train_data_tensor_gray.shape)
print("train bin dimensions ", train_data_tensor_bin.shape)
print("train labels ", len(labels_training))

train gray dimensions torch.Size([784, 600])
train bin dimensions  torch.Size([784, 600])
train labels  600


### 2. Train Model

In [625]:
def evaluate_gaussian(mu, sigma, x):
    coef_norm = 1 / (torch.sqrt(2.0 * torch.pi * sigma ** 2))
    return coef_norm * torch.exp(-0.5*((x - mu)/sigma)**2)

def evaluate_gaussian_log(mu, sigma, x_tensor):
    M = x_tensor.shape[1]
    summation = -1 / (2.0 * torch.pi * sigma ** 2) * ((x_tensor - mu) ** 2).sum(dim=1).view(x_tensor.shape[0], -1)
    p_gaussian = summation - (M/2 * torch.log(torch.tensor(2 * torch.pi))) - (M * torch.log(sigma))
    return p_gaussian
    
def train_model(train_data_tensor_bin, train_data_tensor_gray, labels_bin, labels_gray, num_classes = 10):

    first_tensor = True
    p_t_tensor_acc = None
    p_m_1_given_k_acc = None
    p_m_0_given_k_acc = None
    mu_given_k_acc = None
    sigma_given_k_acc = None
    #gaussian_given_k_acc = None
    #print(train_data_tensor_gray[650, 0:60])
    
    for k in range(num_classes):
        # Filtra train_data_tensor por clase para dataset binarizado y en escala de grises
        train_data_tensor_bin_per_k = train_data_tensor_bin[:, labels_bin == k].type(torch.int64)
        train_data_tensor_gray_per_k = train_data_tensor_gray[:, labels_gray == k].type(torch.float64)
        #print(train_data_tensor_bin_per_k)
        #print(train_data_tensor_gray_per_k)
        
        # D = cantidad de filas (784 pixeles)
        # N = cantidad de columnas (600 imágenes)
        D, N = train_data_tensor_bin.shape
        
        # Estimacion de probabilidad a priori
        p_t_tensor = torch.tensor([train_data_tensor_bin_per_k.shape[1] / N])
        #print(p_t_tensor)
        
        # Estimacion de verisimilitud para cada pixel por clase
        p_m_1_given_k = ((train_data_tensor_bin_per_k == 1).sum(dim=1)/train_data_tensor_bin_per_k.shape[1])+1e-12
        #print(p_m_pix_val_given_k)
        
        # Transforma tensor de verisimilitud a una sola columna
        p_m_1_given_k = p_m_1_given_k.view(p_m_1_given_k.shape[0], -1)
        #print(p_m_pix_val_given_k)
    
        # Calculo de media y desviacion estandar
        mu_given_k = torch.mean(train_data_tensor_gray_per_k, dim=1)
        sigma_given_k = torch.std(train_data_tensor_gray_per_k, dim=1)+1e-12
        #print(mu_given_k.shape)
        #print(sigma_given_k.shape)
        
        # Transforma tensor de mu y sigma a una sola columna
        mu_given_k = mu_given_k.view(mu_given_k.shape[0], -1)
        sigma_given_k = sigma_given_k.view(sigma_given_k.shape[0], -1)
        #print(mu_given_k)
        
        # Estimacion de funcion de densidad de probabilidad
        #gaussian_given_k = evaluate_gaussian_log(mu_given_k, sigma_given_k, train_data_tensor_gray_per_k)
        #print(gaussian_given_k)
    
        if(first_tensor):
            first_tensor = False
            p_m_1_given_k_acc = p_m_1_given_k
            p_t_tensor_acc = p_t_tensor
            mu_given_k_acc = mu_given_k
            sigma_given_k_acc = sigma_given_k
            #gaussian_given_k_acc = gaussian_given_k
        else:
            p_m_1_given_k_acc = torch.cat((p_m_1_given_k_acc, p_m_1_given_k), 1)
            p_t_tensor_acc = torch.cat((p_t_tensor_acc, p_t_tensor), 0)
            mu_given_k_acc = torch.cat((mu_given_k_acc, mu_given_k), 1)
            sigma_given_k_acc = torch.cat((sigma_given_k_acc, sigma_given_k), 1)
            #gaussian_given_k_acc = torch.cat((gaussian_given_k_acc, gaussian_given_k), 1)

    # Saca complemento de p_m_1_given_k_acc
    p_m_0_given_k_acc = 1 - p_m_1_given_k_acc
    
    #print("p_m_0_given_k_acc = ", p_m_0_given_k_acc.shape)
    #print("p_m_1_given_k_acc = ", p_m_1_given_k_acc.shape)
    #print("p_t_tensor_acc = ", p_t_tensor_acc.shape)
    #print("mu_given_k_acc = ", mu_given_k_acc.shape)
    #print("sigma_given_k_acc = ", sigma_given_k_acc.shape)
    #print("gaussian_given_k_acc = ", gaussian_given_k_acc.shape)
        
    return (list([p_m_0_given_k_acc, p_m_1_given_k_acc]), mu_given_k_acc, sigma_given_k_acc, p_t_tensor_acc)
            
        
#p_m_pix_val_given_k, mu_all, sigma_all, p_t_tensor = train_model(train_data_tensor_bin, train_data_tensor_gray, labels_training)
        
#train model by calculating the prior probabilities
#(p_m_pix_val_given_k, p_t_tensor) = train_model(train_data_tensor, labels_training)
#print("p_m_pix_val_given_k size ", p_m_pix_val_given_k.shape)

### 3. Test Bernoulli Model Function

In [614]:
def test_model_bernoulli(input_torch, p_m_pix_val_given_k, p_t_tensor, num_classes = 10):
    #assumes that the input comes in a row    
    input_torch_single_column = input_torch.view(input_torch.shape[0], -1)
    
    p_t_k_given_m = [] 
    for k in range(num_classes):
        # Extraer probabilidades de acuerdo al input_torch nuevo para cuando pixel es 0
        #print(p_m_pix_val_given_k[0][:,k].shape)
        #print(p_m_pix_val_given_k[0][:,k].unsqueeze(dim=1).shape)
        #print(input_torch_single_column.shape)
        p_m_0_extracted = torch.log(p_m_pix_val_given_k[0][:,k].unsqueeze(dim=1)[input_torch_single_column == 0])
        #p_m_0_extracted = torch.log(p_m_pix_val_given_k[0][:,k][input_torch[k,:] == 0])
        # Estima log para todas las probabilidades extraidas
        p_m_0_estimated = p_m_0_extracted.sum(dim=0) 
        # Extraer probabilidades de acuerdo al input_torch nuevo para cuando pixel es 1
        p_m_1_extracted = torch.log(p_m_pix_val_given_k[1][:,k].unsqueeze(dim=1)[input_torch_single_column == 1])
        #p_m_1_extracted = torch.log(p_m_pix_val_given_k[1][:,k][input_torch[k,:] == 1])
        # Estima log para todas las probabilidades extraidas
        p_m_1_estimated = p_m_1_extracted.sum(dim=0) 
        # Estimacion de la probabilidad posterior
        p_t_k_given_m.append(p_m_0_estimated + p_m_1_estimated + torch.log(p_t_tensor[k]))
        
    scores_classes = p_t_k_given_m
    #predicted_label = max(p_t_k_given_m)
    predicted_label = scores_classes.index(max(scores_classes))
    return (predicted_label, scores_classes)#ocupo retorna la clase 

new_image_ber = load_single_image(img_index = 65, mode="bin") #Numero 1
#input_torch = torch.bernoulli(new_image)
#print(new_image_ber.shape)
test_model_bernoulli(new_image_ber, p_m_pix_val_given_k, p_t_tensor, num_classes = 10)

#(predicted_label, scores_classes) = test_model(train_data_tensor[:, 500], p_m_pix_val_given_k, p_t_tensor)
#print("predicted_label ", predicted_label)
#print("real label ", labels_training[500])
#acc = test_model_batch(train_data_tensor, labels_training, p_m_pix_val_given_k, p_t_tensor)
#print("Model accuracy ", acc)
    


(1,
 [tensor(-451.0027),
  tensor(-72.5437),
  tensor(-183.4687),
  tensor(-284.4493),
  tensor(-281.4951),
  tensor(-161.0242),
  tensor(-266.5751),
  tensor(-228.2547),
  tensor(-169.1813),
  tensor(-220.9562)])

### 4. Test Gaussian Model Function

In [652]:
def test_model_gaussian(input_torch, mu, sigma, p_t_tensor, num_classes = 10):
    #assumes that the input comes in a row
    input_torch_reshaped = input_torch.view(input_torch.shape[0], -1)
    #print(input_torch_reshaped.shape)
    
    # Estimacion de la probabilidad de verisimilitud para todas las clases
    p_gaussian_estimated = evaluate_gaussian(mu, sigma, input_torch_reshaped)+1e-12
    #print(p_gaussian_estimated.shape)

    p_posterior_all = []
    for k in range(num_classes):
        # Estimacion de la probabilidad posterior
        p_posterior_all.append(torch.log(p_gaussian_estimated[:, k]).sum(dim=0) + torch.log(p_t_tensor[k]))
        #p_posterior_all.append(p_gaussian_estimated[:, k].sum(dim=0) + torch.log(p_t_tensor[k]))
        
    scores_classes = p_posterior_all
    predicted_label = scores_classes.index(max(scores_classes))
    #predicted_label = max(scores_classes)
       
    return(predicted_label, scores_classes)

new_image_gaus = load_single_image(img_index = 65, mode="gray") #Numero 1
#print(new_image_gaus)
test_model_gaussian(new_image_gaus, mu_all, sigma_all, p_t_tensor, num_classes = 10)

(1,
 [tensor(-1638.3760, dtype=torch.float64),
  tensor(5159.8349, dtype=torch.float64),
  tensor(-911.2691, dtype=torch.float64),
  tensor(23.9232, dtype=torch.float64),
  tensor(-2155.0854, dtype=torch.float64),
  tensor(-1715.8437, dtype=torch.float64),
  tensor(1761.4833, dtype=torch.float64),
  tensor(-898.4278, dtype=torch.float64),
  tensor(-1119.1777, dtype=torch.float64),
  tensor(1160.7031, dtype=torch.float64)])

### 6.Test Bernoulli Model batch Function: Accuracy

In [599]:
def test_Bernoulli_model_batch(test_set, labels, p_m_pix_val_given_k, p_t_tensor):
    
    number_observations = test_set.shape[0]
    number_correct_predictions = 0
    for observation_id in range(number_observations):
        current_observation = test_set[observation_id]
        predicted_label,scores_classes = test_model_bernoulli(current_observation, p_m_pix_val_given_k, p_t_tensor, num_classes = 10)
        if (predicted_label == labels[observation_id]): #poner la clase label= clase 
            number_correct_predictions += 1

    accuracy = number_correct_predictions/number_observations
    #print(accuracy)
    return accuracy


### 5.Test Gaussian Model batch Function: Accuracy

In [600]:
def test_Gaussian_model_batch(test_set, labels, mu_tensor, sigma_tensor, p_t_tensor):
    
    number_observations = test_set.shape[0]
    number_correct_predictions = 0
    for observation_id in range(number_observations):
        current_observation = test_set[observation_id]
        predicted_label,scores_classes = test_model_gaussian(current_observation, mu_tensor, sigma_tensor, p_t_tensor, num_classes = 10)
        if (predicted_label == labels[observation_id]): #poner la clase label= clase 
            number_correct_predictions += 1

    accuracy = number_correct_predictions/number_observations
    #print(accuracy)
    return accuracy


## *II Sección: Prueba del Modelo*

### 1.Train Model with 100% dataset

In [653]:
#Transpose dimensions to be processed by train_test_split from [784,600] to [600,784]
dataset_bin = torch.transpose(train_data_tensor_bin, 0, 1)
dataset_gray = torch.transpose(train_data_tensor_gray, 0, 1)
#print(dataset_bin.shape, labels_training.shape)

#Training model with 100% of the dataset
p_m_pix_val_given_k, mu_all, sigma_all, p_t_tensor = train_model(train_data_tensor_bin, train_data_tensor_gray, labels_training, labels_training)

#Get accuraccy for Bernoulli
bernoulli_accuracy = test_Bernoulli_model_batch(dataset_bin, labels_training, p_m_pix_val_given_k, p_t_tensor)
print("Bernoulli Accuracy:",bernoulli_accuracy*100,"%")

#Get accuraccy for Gaussian
gaussian_accuracy = test_Gaussian_model_batch(dataset_gray, labels_training, mu_all, sigma_all, p_t_tensor)
print("Gaussian Accuracy:",gaussian_accuracy*100,"%")

Bernoulli Accuracy: 91.5 %
Gaussian Accuracy: 17.0 %


### 2.Train Model with 70% dataset and Test Model with 30% dataset

In [654]:
# Creando particiones 70% Training y 30% Testing para el set de datos
features_train_bin, features_test_bin, label_train_bin, label_test_bin = train_test_split(dataset_bin, labels_training, test_size=0.3,random_state=9)
features_train_gray, features_test_gray, label_train_gray, label_test_gray = train_test_split(dataset_gray, labels_training, test_size=0.3,random_state=9)
#print(features_train_bin.shape, features_test_bin.shape)
#print(label_train_bin.shape, label_test_bin.shape)

#Revert transpose dimensions for train model
x_train_bin = torch.transpose(features_train_bin, 1, 0)
x_test_bin = torch.transpose(features_test_bin, 0, 1)
x_train_gray = torch.transpose(features_train_gray, 0, 1)
x_test_gray = torch.transpose(features_test_gray, 0, 1)
#print(x_train_bin.shape, x_test_bin.shape)
#print(label_train_bin.shape, label_test_bin.shape)
#print(x_train_gray.shape, x_test_gray.shape)
#print(label_train_gray.shape, label_test_gray.shape)

#Training model with 70% of the dataset
p_m_pix_val_given_k, mu_all, sigma_all, p_t_tensor = train_model(x_train_bin, x_train_gray, label_train_bin, label_train_gray)

#Get accuraccy for Bernoulli
bernoulli_accuracy = test_Bernoulli_model_batch(features_test_bin, label_test_bin, p_m_pix_val_given_k, p_t_tensor)
print("Bernoulli Accuracy:",bernoulli_accuracy*100,"%")

#Training model with 70% of the dataset
p_m_pix_val_given_k, mu_all, sigma_all, p_t_tensor = train_model(x_train_bin, x_train_gray, label_train_bin, label_train_gray)

#Get accuraccy for Gaussian
gaussian_accuracy = test_Gaussian_model_batch(features_test_gray, label_test_gray, mu_all, sigma_all, p_t_tensor)
print("Gaussian Accuracy:",gaussian_accuracy*100,"%")

Bernoulli Accuracy: 81.11111111111111 %
Gaussian Accuracy: 18.88888888888889 %
